<img src="https://github.com/alan-barzilay/NLPortugues/blob/master/imagens/logo_nlportugues.png?raw=true"  style="height:65%" align="right">


# Lista 6 - LSTM&GRU 
**Nome:**

**Numero Usp:**









______________




O objetivo desta lista é fazer com que vocês treinem um modelo de análise de sentimentos utilizando GRU's e LSTM's. Essa lista é semelhante a lista 03 onde aprendemos a usar embeddings e onde você ja recebeu a arquitetura do seu modelo quase pronta. A diferença é que desta vez você ira construir sozinho sua rede e utilizará as camadas que acabamos de aprender: LSTM e GRU.
 
Essa tambêm será a primeira rede recorrente que montaremos, portanto a tokenização será ligeiramente diferente (por exemplo o padding não é mais necessário.)

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
tf.__version__
# '2.2.0-rc3'

'2.2.0-rc3'

## Importando os dados como um dataframe

Para esta lista nós utilizaremos um recorte de 10 mil linhas do dataset **B2W-Reviews01** que consiste em avaliações de mais de 130k compras online no site Americanas.com e [esta disponivel no github](https://github.com/b2wdigital/b2w-reviews01) sob a licensa CC BY-NC-SA 4.01.

In [4]:
b2wCorpus = pd.read_csv("../data/b2w-10k.csv")
b2wCorpus.head()

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958,F,RJ,NaN,NaN,NaN,NaN,NaN
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996,M,SC,NaN,NaN,NaN,NaN,NaN
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984,M,SP,NaN,NaN,NaN,NaN,NaN
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985,F,SP,NaN,NaN,NaN,NaN,NaN
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994,M,MG,NaN,NaN,NaN,NaN,NaN


In [5]:
b2wCorpus["review_text"]

0       Estou contente com a compra entrega rápida o ú...
1       Por apenas R$1994.20,eu consegui comprar esse ...
2       SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...
3       MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...
4       A entrega foi no prazo, as americanas estão de...
                              ...                        
9994    Celular muito rápido, com processador e armaze...
9995    achei o produto muito frágil, o material veio ...
9996    Uma porcaria pois ñ recebi ñ recomendo pra nin...
9997    Maquina excelente,super pratica. recomendo.ent...
9998    Agradeço pelo compromisso, obrigado. ,...........
Name: review_text, Length: 9999, dtype: object


## Pré-processamento 
# <font color='blue'>Questão 1 </font>
Copie suas etapas de préprocessamento da lista 03, ou seja, selecione apenas as colunas relevantes ("review_text" e "recommend_to_a_friend"), converta a coluna "review_text" de uma coluna de `str` para uma coluna de `int` e separe os dados em teste e treino.


In [9]:
# Seu código aqui

df = b2wCorpus[['review_text','recommend_to_a_friend']]
df['recommend_to_a_friend'] = df['recommend_to_a_friend'].map({'Yes': 1, 'No': 0})


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['review_text'], df['recommend_to_a_friend'], test_size=0.20, random_state=17)
# [X_train, y_train]

# df.head()
# X_train

/mnt/dados/home/wseidel/envs/usp-MAC5725b/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


8674    Entrega rápida! E o produto de excelente quali...
8024    recebi meu produto muito rapido, adorei o prod...
4083    contudo, só não comprei porque aprecio ver alg...
4089    Comprei com capa protetora e foi enviado um le...
6420    Excelente produto, cumpre o que promete..........
                              ...                        
8470    Pra fotos ele é ÓTIMO  PARA AS DEMAIS FUNÇÕES ...
6061    produto normal umo produto como tantos outas  ...
9529    Comprei duas unidades, quando usei um logo que...
406     O produto é até bom, mas a foi pessíma no aten...
2191    O produto não tem muito a ver com a imagem ilu...
Name: review_text, Length: 7999, dtype: object

## Tokenizando




# <font color='blue'>Questão 2 </font>
Utilizando a camada [`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization) tokenize os inputs.
Declare a camada e então chame a função `adapt()` no seu conjunto de treino para adequar o seu vocabulário aos reviews. 

Note que o uso de padding não é mais necessario.

In [16]:
X_train.shape

(7999,)

In [179]:
# Seu código aqui

VOCAB_SIZE = 5000
MAX_SENTENCE_LENGTH = 75
EMBEDDING_LENGTH = 128
QNT_EPOCAS_A_TESTAR = 1

vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_sequence_length=MAX_SENTENCE_LENGTH,  # Only valid in INT mode.
)

vectorize_layer.adapt(X_train.values)

In [180]:
# https://towardsdatascience.com/you-should-try-the-new-tensorflows-textvectorization-layer-a80b3c6b00ee
# https://stackoverflow.com/questions/63207118/unable-to-concatenate-features-to-same-shape

from tensorflow.keras import layers
input_ = layers.Input(shape=(1,), dtype=tf.string)

model = tf.keras.Sequential()
model.add(input_)
model.add(vectorize_layer)
model.predict(["Eu gostei de passear pela casa ontem daskdaslçlk"])
# model.predict(
# [ 
#   ["Entrega rápida! E o produto de"],
#   ["klksa kkjhdjs oppoi dndasd "]
# ]).shape

array([[ 32,  44,   4,   1, 104, 189, 944,   1,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])

## LSTM&GRU

Agora vamos juntar a camada do tokenizador a nossa camada [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) e definir o resto de nosso modelo.

#  <font color='blue'>Questão 3 </font>

a) Defina, compile, treine e avalie seu modelo, utilize camadas  [LSTM](https://keras.io/api/layers/recurrent_layers/lstm/).
Atenção a dimensão do input da camada de embedding, lembre se que < OOV > e < PAD > possuem seus próprios tokens.
 
 
 
b) Como foi a performance desta rede em comparação a da lista 3?




**<font color='red'> Sua resposta aqui </font>**

In [181]:
# Seu código aqui
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

from tensorflow.keras import layers
import random

model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    vectorize_layer,
    tf.keras.layers.Embedding(VOCAB_SIZE+1, output_dim=EMBEDDING_LENGTH, input_length=MAX_SENTENCE_LENGTH),
    tf.keras.layers.LSTM(
        units=EMBEDDING_LENGTH, 
        activation="tanh",
        recurrent_activation="sigmoid",
    ),
    

#     layers.Dense(128, activation='relu'),
    layers.Dropout(0.4, seed=random.randint(0,15)),
    
    layers.Dense(units=1, activation='sigmoid'),
])
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["acc"])

model.fit(X_train, y_train,epochs=QNT_EPOCAS_A_TESTAR)

model.summary()

loss, accuracy = model.evaluate(x=X_test,y=y_test)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/5
250/250 [==============================] - 18s 73ms/step - loss: 0.4982 - acc: 0.7525
Epoch 2/5
250/250 [==============================] - 17s 68ms/step - loss: 0.4412 - acc: 0.7920
Epoch 3/5
250/250 [==============================] - 19s 74ms/step - loss: 0.4358 - acc: 0.7975
Epoch 4/5
250/250 [==============================] - 17s 67ms/step - loss: 0.4707 - acc: 0.7671ETA: 1
Epoch 5/5
250/250 [==============================] - 18s 71ms/step - loss: 0.4896 - acc: 0.7700
Model: "sequential_76"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_30 (TextV (None, 63)                0         
_________________________________________________________________
embedding_44 (Embedding)     (None, 63, 128)           640128    
_________________________________________________________________
lstm_38 (LSTM)               (None, 128)               131584    
_____________________________

#  <font color='blue'>Questão 4 </font>

a) Defina, compile, treine e avalie seu modelo, utilize camadas [GRU](https://keras.io/api/layers/recurrent_layers/gru/).
Atenção a dimensão do input da camada de embedding, lembre se que < OOV > e < PAD > possuem seus próprios tokens.
 
 
 
b) Como foi a performance desta rede em comparação a da lista 3?


**<font color='red'> Sua resposta aqui </font>**

In [183]:
# Seu código aqui
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

from tensorflow.keras import layers
model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    vectorize_layer,
    tf.keras.layers.Embedding(VOCAB_SIZE+1, output_dim=EMBEDDING_LENGTH, input_length=MAX_SENTENCE_LENGTH),
    tf.keras.layers.GRU(
        units=EMBEDDING_LENGTH, 
        activation="tanh",
        recurrent_activation="sigmoid",
    ),
    layers.Dropout(0.4, seed=random.randint(0,15)),
    layers.Dense(units=1, activation='sigmoid'),
])
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["acc"])

model.fit(X_train, y_train,epochs=1)

model.summary()

loss, accuracy = model.evaluate(x=X_test,y=y_test)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

250/250 [==============================] - 15s 59ms/step - loss: 0.5606 - acc: 0.7477
Model: "sequential_78"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_30 (TextV (None, 63)                0         
_________________________________________________________________
embedding_46 (Embedding)     (None, 63, 128)           640128    
_________________________________________________________________
gru_2 (GRU)                  (None, 128)               99072     
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_54 (Dense)             (None, 1)                 129       
Total params: 739,329
Trainable params: 739,329
Non-trainable params: 0
_________________________________________________________________
63/63 [====================

## Redes Bi-direcionais
#  <font color='blue'>Questão 5 </font>

a) Defina, compile, treine e avalie um novo modelo que utilize contexto em ambas as direções usando a camada [`Bidirectional()`](https://keras.io/api/layers/recurrent_layers/bidirectional/), seja com camadas GRU ou LSTM.


b) Como foi sua performance em relação as questões anteriores com contexto unidirecional?

**<font color='red'> Sua resposta aqui </font>**

In [ ]:
# Seu código aqui
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

from tensorflow.keras import layers
model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    vectorize_layer,
    tf.keras.layers.Embedding(VOCAB_SIZE+1, output_dim=EMBEDDING_LENGTH, input_length=MAX_SENTENCE_LENGTH),
    tf.keras.layers.GRU(
        units=EMBEDDING_LENGTH, 
        activation="tanh",
        recurrent_activation="sigmoid",
    ),
    layers.Dropout(0.4, seed=random.randint(0,15)),
    layers.Dense(units=1, activation='sigmoid'),
])
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["acc"])

model.fit(X_train, y_train,epochs=1)

model.summary()

loss, accuracy = model.evaluate(x=X_test,y=y_test)

print("Loss: ", loss)
print("Accuracy: ", accuracy)